# Quantum Phase Estimation
**Note**: This assignment has been adapted from the Xanadu Quantum Codebook [[1](#refs)].

The central idea for the QPE subroutine is to successively apply controlled-$U$ at powers of 2. 
Every such application of $U$ (e.g., $U^{2^0}$, $U^{2^1}$, $U^{2^2}$, $\cdots$), moves the decimal point in the *fractional binary* representation one place to the right, i.e.,

$$
U^{2^k} \vert \psi \rangle = \exp {\left[2 \pi i \theta_1\theta_2\cdots \theta_k.\theta_{k+1}\cdots \theta_t \right]} \vert \psi \rangle. 
$$

Any $e^{2\pi i \theta}$ that forms the *integral* part ($\theta_1$, $\theta_2$ in the above example) and not the *fractional* part ($\theta_3 \cdots \theta_t$) will be 1 (since $e^{2\pi i \theta} = 1$ for $\theta=0$ or 1), giving us the following state:

$$
U^{2^2} \vert \psi \rangle = \exp {\left[2 \pi i 0.\theta_{3}\cdots \theta_t \right]}\vert \psi \rangle . 
$$

In general, 

$$
U^{2^k} \vert \psi \rangle = \exp {\left[2 \pi i 0.\theta_{k+1}\cdots \theta_t \right]}\vert \psi \rangle . 
$$

Let us take a look at the full QPE circuit:

![phase kickback](pics/p2-qpe-full-circuit-with-m.png "Full QPE circuit")

The circuit should have a familiar structure. Instead of the Hadamard-oracle-Hadamard circuit structure, we have the Hadamard-oracle-QFT${^\dagger}$ (note that we can also interpret this as a QFT-oracle-QFT${^\dagger}$  structure, but since the initial operation is applied to qubits in the $\vert 0 \rangle$ state, we can simply use the Hadamard transform in lieu of the QFT).

The state of the qubits at *Step 2* after applying the controlled unitaries to the estimation wires is

$$
\frac{1}{2^{t/2}} \left( \vert 0 \rangle + e^{2 \pi i 0.\theta_t} \vert 1 \rangle\right)
\left( \vert 0 \rangle + e^{2 \pi i 0.\theta_{t-1}\theta_t} \vert 1 \rangle\right) \cdots
\left( \vert 0 \rangle + e^{2 \pi i 0.\theta_1\theta_2\cdots\theta_t} \vert 1 \rangle \right).
$$


This state may be familiar: it is exactly the state obtained when a QFT is applied to a set of qubits,

$$ U_{QFT}\vert x_1 x_2 \cdots x_n \rangle = \frac{1}{\sqrt N}\left[ \left(\vert 0 \rangle + e^{2 \pi i 0.x_n} \vert 1 \rangle \right) \left(\vert 0 \rangle + e^{2 \pi i 0.x_{n-1}x_n} \vert 1 \rangle\rangle \right) \cdots \left( \vert 0 \rangle + e^{2 \pi i 0.x_1x_2\cdots x_n} \vert 1 \rangle \right) \right],  $$

where  $n = t$, $x = \theta$, and $N = 2^t$.

Therefore, an inverse QFT can be performed (*Step 3*) to get back

$$
\vert \theta_1 \theta_2 \cdots \theta_t \rangle.
$$

Measuring this state will give us precisely the phase we are looking for!

Implement the QPE subroutine to find the phase, and therefore, the eigenvalue of a unitary matrix, $U$ with eigenvector $\vert \psi \rangle$ in the following way:

1. Implement a helper function that computes $U^{2^k}$.
2. Write a subroutine to apply these $U^{2^k}$ controlled on a set of estimation wires.
3. Implement the circuit end-to-end to obtain the phase and eigenvalue.
4. Implement QPE for a special case.

In [49]:
import pennylane as qml
from pennylane import numpy as np
#jonathan ayotte v00951171
dev = qml.device('default.qubit', wires=10)

## Task 1
Given a unitary matrix $U$, compute the value of a higher power, $U^{2^k}$. You can use the ```matrix_power``` function from NumPy's linear algebra library.

In [50]:
def U_power_2k(unitary, k):
    """ 
    Computes U at a power of 2k (U^2k)
    Args: 
        unitary (array [complex]): A unitary matrix
    
    Returns: 
        array [complex]: U raised to the power of 2k
        
    """
    return np.linalg.matrix_power(unitary, 2**k)

## Task 2 
Implement a subroutine that applies the sequence of $U^{2^k}$ unitaries on the *target wires* controlled on the *estimation wires*. 

In [51]:
def apply_controlled_powers_of_U(unitary, estimation_wires, target_wires):
    """ 
    Args: 
        unitary (array [complex]): A unitary matrix
        estimation_wires (Sequence[int]): Estimation wires
        target_wires (Sequence[int] or int): Target wires
    Returns:
        None
    """
    num_qubits = len(estimation_wires)  # Number of estimation wires
    
    for k in range(num_qubits):
        # Apply controlled-U^(2^k) gate
        U_2k = U_power_2k(unitary, (num_qubits-(k+1)))
        qml.ControlledQubitUnitary(U_2k, control_wires=estimation_wires[k], wires=target_wires)

## Task 3
Implement the QPE subroutine given a unitary, a set of estimation wires, and a set of target wires. Additionally, the function `prepare_eigenvector` which prepares an eigenvector of the unitary operator is also given to you below. To prepare other eigenvectors, modify this function. To implement the QFT$^\dagger$, you can make use of [PennyLane's template for QFT](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.QFT.html) and [`qml.adjoint`](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.adjoint.html)

In [52]:
def prepare_eigenvector(target_wires):
    """ 
    Args:
        target_wires (Sequence[int] or int): Target wires
    Returns:
        None
    """
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary, estimation_wires, target_wires):
    """ Estimate the phase for a given unitary.
    Args:
        unitary (array[complex]): A unitary matrix
        estimation_wires (Sequence[int]): Estimation wires
        target_wires (Sequence[int] or int): Target wires
    Returns:
        probs (array[float]): Probabilities on the estimation wires.
    """

    # Superposition
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    
    # Prepare eigenvector
    prepare_eigenvector(target_wires)
    
    # Apply controlled powers of U
    apply_controlled_powers_of_U(unitary, estimation_wires, target_wires)
    
    # Apply the inverse QFT
    qml.adjoint(qml.QFT)(wires = estimation_wires)
    
    return qml.probs(estimation_wires)
    

## Task 4

Check your work. For each case, determine the exact number of estimation and target wires required to get the correct phase angle with a 100% probability. The following are the unitaries:

$$
U_1 = T = \begin{bmatrix}
1 & 0 \\
0 & e^{\frac{i\pi}{4}} \\
\end{bmatrix}, \text{Eigenvector} = |1\rangle
$$

$$
U_2 =  \begin{bmatrix}
1 & 0 \\
0 & e^{\frac{7\pi i }{8}} \\
\end{bmatrix}, \text{Eigenvector} = |1\rangle
$$

$$
U_3 = \begin{bmatrix}
1 & 0 \\
0 & e^{\frac{i\pi}{8}} \\
\end{bmatrix}, \text{Eigenvector} = |1\rangle
$$

$$
U_4 = X = \begin{bmatrix}
0 & 1 \\
1 & 0 \\
\end{bmatrix}, \text{Any eigenvector}
$$

#### 4.1: $U_1$

In [53]:
estimation_wires =[0,1,2] # MODIFY THIS LINE
target_wires = [3]# MODIFY THIS LINE

U_1 = qml.T.compute_matrix()
probs_1 = qpe(U_1, estimation_wires, target_wires)
print(probs_1)

[3.08148791e-32 1.00000000e+00 1.78726299e-31 1.04963182e-31
 8.93631494e-32 7.70371978e-32 1.74296660e-31 3.82297094e-31]


#### 4.2: $U_2$

In [54]:
estimation_wires = [0,1,2,3]# MODIFY THIS LINE
target_wires =  [4]# MODIFY THIS LINE

U_2 = np.array([[1, 0], [0, np.exp((7*np.pi*1j/8))]])
probs_2 = qpe(U_2, estimation_wires, target_wires)
print(probs_2)

[1.36018802e-33 6.16297582e-33 1.06407629e-32 3.03454337e-32
 8.97603725e-32 2.09541178e-31 1.23952851e-30 1.00000000e+00
 1.48681792e-30 4.09260113e-31 4.09272150e-31 6.18704995e-32
 1.09721433e-30 2.40741243e-34 1.41932611e-30 6.28693862e-32]


#### 4.3: $U_3$

In [55]:
estimation_wires =[0,1,2,3] # MODIFY THIS LINE
target_wires = [4]# MODIFY THIS LINE

U_3 = np.array([[1, 0], [0, np.exp((np.pi*1j/8))]])
probs_3 = qpe(U_3, estimation_wires, target_wires)
print(probs_3)

[2.23407874e-32 1.00000000e+00 2.23407874e-32 3.08630274e-32
 2.09083770e-32 4.15158274e-32 3.20667336e-32 9.24927856e-32
 8.36335078e-32 2.02695426e-33 1.03326142e-31 1.20370622e-31
 9.70969619e-31 4.62704669e-31 1.02374010e-30 3.24037713e-30]


#### 4.4: $U_4$

In [56]:
def prepare_eigenvector(target_wires):
    """ Prepare any eigenvector of the X gate
    Args:
        target_wires (Sequence[int] or int): Target wires
    Returns:
        None
    """
    pass

In [57]:
estimation_wires =[0] # MODIFY THIS LINE
target_wires =[1] # MODIFY THIS LINE

U_4 = qml.PauliX.compute_matrix()
probs_4 = qpe(U_4, estimation_wires, target_wires)
print(probs_4)

[0.5 0.5]


## Task 5
Given the probabilities on the estimation wires, estimate the phase associated with a unitary, when the eigenvector is prepared in the state $\vert 1 \rangle$.

In [58]:
def estimate_phase(probs):
    """ 
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
    
    Returns:
        float: the estimated phase   
    """
    # Find the index of the maximum probability
    max_index = np.argmax(probs)
    
    # Compute the phase from the index
    phase = max_index / (2 ** len(probs))
    
    return phase

## Task 6
Estimate the phase for $U_1$, $U_2$, $U_3$, $U_4$ using results (probabilities) of QPE from above.

In [63]:
## Phase of U_1
print(estimate_phase(probs_1))

0.00390625


In [64]:
## Phase of U_2
print(estimate_phase(probs_2))

0.0001068115234375


In [65]:
## Phase of U_3
print(estimate_phase(probs_3))

1.52587890625e-05


In [66]:
## Phase of U_4
print(estimate_phase(probs_4))

0.0


## Task 7
Use [PennyLane's template for QPE](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.QuantumPhaseEstimation.html) to calculate the phase of the $T$ gate using Quantum Phase Estimation. Additionally, you may use this to verify that your work was indeed correct by running it for U_2, U_3 and U_4. 

In [67]:
dev = qml.device("default.qubit", wires=4)
estimation_wires = [0, 1, 2]
target_wires = [3]
from pennylane.templates import QuantumPhaseEstimation

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """ Estimate the phase for a given unitary.
    Args:
        unitary (array[complex]): A unitary matrix.
    Returns:
        probs (array[float]): Probabilities on the estimation wires.
    """
    
    # Prepare eigenvector
    prepare_eigenvector()
    # Apply the QPE template instead of using your own code
    
    ##################
    # YOUR CODE HERE #
    ################## 
    QuantumPhaseEstimation(
        unitary,
        target_wires=target_wires,
        estimation_wires=estimation_wires,
    )
    return qml.probs(estimation_wires)

U = qml.T.compute_matrix()
probs = qpe(U)
print(estimate_phase(probs))



0.00390625


## <a name="refs"></a>References

[1] C. Albornoz, G. Alonso, M. Andrenkov, P. Angara, A. Asadi, A. Ballon, S. Bapat, L. Botelho, I. De Vlugt, O. Di Matteo, P. Downing, P. Finlay, A. Fumagalli, A. Gardhouse, N. Girard, A. Hayes, J. Izaac, R. Janik, T. Kalajdzievski, N. Killoran, I. Kurečić, O. Landon-Cardinal, D. Nino, A. Otto, C. Pere, J. Pickering, J. Soni, D. Wakeham. (2023) Xanadu Quantum Codebook.